## 5.3 趋势交易

动量效应是动量策略的基础。**动量效应(Momentum Effect)**又称"惯性效应"，指资产价格具有延续原来运动方向的趋势，也就是说，过去一段时间收益较高的资产在未来仍将会获得较高的收益，因此投资者可以通过购买历史表现好的资产、卖出表现差的资产，获得低成本的超额收益。
利用这一效应进行交易的策略就叫动量交易策略。本小节将介绍动量交易策略的方法论、构造算法及其实现和实证结果展示。

### 5.3.1 方法论

实施动量策略的所有步骤可以用以下流程图概括。


<img src="5-方法论.png" width="500" />

#### 5.3.1.1 选择商品期货

$T+0、T+1$或$T+N$都是交易中的交割政策，代表双方的标的和资金的交割发生在同一天、下一个交易日或下一个交易日 $N$ 交易日。 如果实行$T+1$交易系统，交易量将比$T+0$交易系统小很多，因为同样数量的资金一天可以多次进出，大大提高了交易速度。 相同数量的钱可以移动。

由于与中国股市相比，期货的资金容量相对较小。 因此，中国期货$T+0$交易系统的核心目的是激活期货市场的交易。 由于重要的是选择流动性高的商品期货，过滤掉流动性低的商品，本文选择$T+0$合约的商品。

#### 5.3.1.2 展期

国内商品期货是有期限的，即不能无限期持有，必须在交割月前平仓。 否则交易所将强制平仓。 因此，为了获得完整的未来价格时间序列，我们需要对期货合约进行展期。

首先，我们收集所有最接近和次接近的合约及其未来价格。 然后，我们在到期前一个月持有第一个最近的合约。 在该月月底，头寸应展期至第二个最接近的合约，并持有至多一个月。 然后重复此过程，最终我们将获得一系列未来回报。

<img src="5-展期.png" width="600" />

#### 5.3.1.3 计算对数收益

在每个频率周期结束时，我们对期货合约进行排序，并根据过去的排名周期收益对其进行排名，其中每种商品的收益计算为对数百分比变化，如下所示：

<img src="5-log1.png" width="600" />

其中，
* $R_{i,F}$：对数收益的百分比变化；
* $C_{i,t_{1}}$：当前收盘价$t_1$；
* $C_{i,t_{0}}$：当前收盘价$t_0$；


那么，在频率 F 的 H 个排名周期内，每种商品期货合约的平均收益率为：

<img src="5-log2.png" width="600" />

#### 5.3.1.4 动量效应的假设检验

为了测试动量利润的重要性，我们利用渐近正态分布为 $N(0,1)$ 的 t 统计量，并使用 p 值。 单方面的零假设和备择假设是：

<img src="5-hypothesis.png" width="600" />

其中，

* $\mu$：动量策略的平均收益；
* p-value$<$0.1：动量效应存在于 10% 显着性水平。
* p-value$<$0.05：动量效应存在于 5\% 显着性水平。

交易信号的选择：

<img src="5-交易信号.png" width="600" />

<img src="5-ttest.png" width="600" />

我们的交易信号选择也是基于单边检验的结果：在每个时间点，执行t-test(如上图所示),在动量效应显着时进行交易； 如果没有，保持直到下一个信号存在。

#### 5.3.1.5 滚动方法

如果测试多个长周期动量策略，将排名周期和持有周期从所选时间跨度的起点滚动到终点具有重要意义。以 5 年每周 5-2 动量策略为例（即我们使用 5 周排名期和 2 周持有期 5 年的每周动量策略进行交易）。 

如下图所示，我们通过**从上一个持有期的结束点推后一个持有期的长度来选择下一个排名期的起点**进行滚动。 也就是说，我们总是将上一个持有期的结束点（见图H1）与下一个排名期的结束点（见图R2）对齐。 滚动过程如下：
<img src="5-滚动.jpg" width="650" />

>上图中，我们有5个时间点：A、B、C、D、E。

>第一个排序期是从A到D，第一个持有期是从D到E。从点E向前推5周，即从点B->E，我们得到了第二个排序期(ranking)，即R2；那么第二个持有期(holding )，即从E->F。继续这个过程，我们可以得到5年的所有ranking和holding period。

值得注意的是，上图显示的所有排名时段都有重叠时段。这允许交易者在前一个持有期结束后立即获得新的投资组合组合。规则是，如果假设检验的结果表明 R1 中不存在动量效应，那么我们仍将持有相同的“赢家”和“输家”组合，并且在存在任何动量效应之前不进行交易。例如，如果假设第二个时期确实存在动量效应，那么在 H2 之后，我们将交易投资组合并重新开仓。由于排名周期 R3，从 C 点到 F，与 H2，从 E 到 F 有重叠。因此，虽然在 period2 中测试动量为真，但交易后我们可以立即得到新的组合。

### 5.3.2 构造算法与实现

实施动量策略的步骤如下：
> **Step1:** 确定动量策略的总时间跨度（例如一个月或五年），并确定数据的频率（即分钟/每天/每周/每月）；

> **Step2:** 决定交易的商品；选择标准在 5.3.1.1小节中讨论过;

> **Step3:** 获取产品的连续价格序列；<br/>但是，并不是所有的产品都有连续的收盘价序列。因此，应该使用在 5.3.1.2 小节中讨论的展期(roll-over)方法将价格序列从离散转换为连续；

> **Step4:** 利用连续价格序列，计算所选商品的对数收益；

> **Step5:** 确定排名期(Ranking)和持有期(Holding)的长度;

> **Step6:** 通过对对数收益序列排序并且等分，筛选出赢家(winner)和输家(loser)组合；<br/>更具体的说，在形成期的长度内，对数收益系列按照升序平均分为不同的组合排序。因此，排名最高的组是“赢家”投资组合，反之为“输家”投资组合；我们选择买入(long)赢家组合，卖出(short)输家组合；

> **Step7:** 将赢家和输家组合持有一个持有期，测试动量效应；<br/>如果动量效应不存在，则仍然持有此组合进入下一个排名和持有期。否则，立即交易它们，并同时选择另外一组赢家和输家组合。这部分在上面5.3.1.4小节中讨论。

> **Step8:** 在所选时间范围内滚动动量策略。详细的滚动方法在上面5.3.1.5小节中介绍；

> **Step9:** 重复step1到8，测试排名长短和持有期的组合，哪个组合能带来最好的收益

### 5.3.3 实证结果

动量策略的构建过程遵循 Jegadeesh 和 Titman(1993) 提出的方法，详细过程如上面流程图所示。

本节提供了 16 种动量策略组合的汇总统计数据，选取了形成期 R=2,4,7,9周，排序期 R=2,4,7,9周的动量策略进行测试。

投资组合“MOM R/H”代表动量策略，排名期为 R 周，持有期为 H 周。 

#### 5.3.3.1 单个交易所 -大连商品交易所

动量效应可以通过策略产生的利润来测试。

单边假设检验是：
\begin{equation}
\nonumber
\left\{
\begin{aligned}
H0: \mu = 0 \nonumber \\ 
H1: \mu > 0 \nonumber
\end{aligned}
\right.
\end{equation}

其中，$\mu$是投资组合的预期收益。零假设是投资组合的预期收益小于或等于零，而备择假设是投资组合的预期收益显着大于零。 

我们利用 t 统计量和 p 值来检验我们是否可以拒绝原假设。 如果 p 值小于 0.1，我们可以得出结论，平均每周对数回报是正的，这表明存在动量效应。

样本期为 2015 年 1 月至 2020 年 7 月。

<img src="5-mom1_new.png" width="500" />

>从上面的统计表中，我们可以得出结论，16 个投资组合中有 9 个获得了正利润。 此外，在这 9 个投资组合中，有 7 个的利润大于 0.01，其中 MOM4/4 的收益最高，约为 0.072，而波动性最高，为 0.0633。 

>然而，这些动量策略中的大多数仍然拥有微不足道的利润并且承担的风险较小。可以很容易地得出结论，具有较高回报的投资组合将面临更多风险。然而，这并不总是正确的。例如，从以上统计来看，MOM9/9策略的负收益为-0.1765，但承担的风险因子为0.495，在所有风险中排名最大。

<img src="5-mom2_new.png" width="500" />

>此表得出结论，16 个投资组合中有 7 个具有显着的动量效应，因为 p 值在 10% 显着性水平下小于 0.1。 最成功的动量策略是MOM4/4，平均收益为0.071885208，动量效应的显着性。

#### 5.3.3.2 多个时间段比较

* 5年期: 2015.01 - 2020.07


* 特殊时期：

    * 牛市（The bull market）: 2015.12 - 2017.09
    * 疫情（COVID-19）: 2019.12 - 2020.07



#### 1. 多个交易所5年期比较

和上述过程相同，我们测试了多个交易所的所有产品的动量效应，如下表所示。

<img src="5-full_mom.png" width="400" />

>此表得出结论，16 个投资组合中只有 3 个有正收益，其余动量策略的收益都为负值。 
<br/>所以我们继续测试了5年期的反转策略。

<img src="5-full_reverse.png" width="450" />

>我们发现，反转策略表现得更好，16 个投资组合中除了 3 个负收益的组合，其余动量策略的收益都为正值。 

#### 2. 多个交易所特殊时期比较

市场是复杂多变的，所以我们选取了2015-2017的牛市（The bull market）和 2019-2020的疫情（COVID-19）这两个阶段进行研究。

#### （1）牛市（The Bull Market）：2015.12 - 2017.09

<img src="5-bull_mom.png" width="450" />

>我们发现，反转策略表现得更好，16 个投资组合中有 9 个负收益的组合，所以我们仍然测试了反转策略在此阶段的情况。 

<img src="5-bull_reverse.png" width="450" />

>我们发现，反转策略表现得更好，16 个投资组合中有 10 个正收益的组合，6个负收益组合。 <br/> 这说明在牛市阶段，反转策略比动量策略更显著。

#### （2）疫情（COVID-19）：2019.12 - 2020.07

<img src="5-covid_mom.png" width="450" />

>我们发现，反转策略表现得更好，16 个投资组合中有 12 个正收益的组合，仅有4个负收益组合。 <br/> 这说明在疫情，动量策略是显著的。

### 5.3.4 策略表现

由于绩效指标是测试和衡量策略是否表现良好的重要指标，因此有必要将其纳入我们的分析中。 

绩效指标包含夏普比率、最大退税比率、风险回报率等指标。

本节所涉及的指标基于年化数据，例如年化收益率和年化标准差。 这些指标的计算公式如下（以周频为例）。

#### 5.3.4.1 衡量指标

**1.资产回报（Asset return）**

\begin{equation}
\nonumber
\begin{aligned}
r_{t}=\frac{NaV_{t}-NaV_{t-1}}{NaV_{t-1}} \nonumber
\end{aligned}
\end{equation}

其中$t$是投资组合的当前时间点，$NaV_{t}$ 和 $NaV_{t-1}$ 分别是投资组合在 $t$ 和 $t-1$ 的资产净值。
  
  
  
**2.年化收益率（Annualized return）**

\begin{equation}
\nonumber
\begin{aligned}
R=[(1+r_{1})(1+r_{2})...(1+r_{i})...(1+r_{n})]^{\frac{52}{n} } -1  \nonumber
\end{aligned}
\end{equation}

其中 $n$ 是观察次数，$r_i$ 表示第 $i-th$ 周回报。




**3.年化波动率（Annualized standard deviation/volatility）**

\begin{equation}
\nonumber
\begin{aligned}
\sigma_{p} =\sqrt{\frac{\sum_{i}^{n}(r_{i}-\bar{r} )^{2}}{n-1} }   \nonumber
\end{aligned}
\end{equation}

其中 $\bar{r}$ 表示样本期间每周收益的样本均值。 那么，年化波动率可以表示为

\begin{equation}
\nonumber
\begin{aligned}
\sigma = \sigma_{p} \times \sqrt{52}   \nonumber
\end{aligned}
\end{equation}




**4.夏普比率（Sharpe Ratio）**

\begin{equation}
\nonumber
\begin{aligned}
Sharpe\; Ratio=\frac{R-R_{f}}{\sigma}  \nonumber
\end{aligned}
\end{equation}

其中 $R$ 和 $\sigma$ 表示 n 周内的年化收益率和波动率，而 $R_{f}$ 是银行账户中的一年期无风险存款利率。

更具体地说，夏普比率反映了单位风险基金净值增长率超过无风险收益率的程度。 如果夏普比率为正，说明基金的平均净值增长率超过了计量期内的无风险利率。 因此，在将银行存款利率作为同期无风险利率的情况下，意味着采取动量策略优于银行存款。 夏普比率越高，基金风险单位的风险回报就越高。





**5.最大回撤率（The maximum drawdown）**

\begin{equation}
\nonumber
\begin{aligned}
Maximum\; Drawdown=max(\frac{D_{i}-D_{j}}{D_{i}})  \nonumber
\end{aligned}
\end{equation}

其中$min(t)<i<j<max(t)$，$min(t)$和$max(t)$表示固定时间段$t$的起点和终点。 $D_{i}$ 和 $D_{j}$ 分别代表投资组合在时间点 $i$ 和 $j$ 的 NaV。

最大回撤率是指产品净值在所选周期内前推至任意历史点时达到最低点时回撤率的最大值。 它用于描述投资产品或投资组合后可能出现的最坏情况。 在某些情况下，最大回撤率比波动率更重要。

根据Chekhlov et al. (2005)的说法，50% 的回撤是不能容忍的。另外，如果最大回撤率超过 20%，账户很可能被关闭，如果在 15%，则会发出警告(Chekhlov et al., 2005)。


#### 5.3.4.2 单个交易所对比

>就交易次数而言：MOM2/2, MOM9/2 都有$>$ 10的次数, 但年化收益率和夏普比率均为负；

>有三个交易策略具有年化正回报：MOM2/9(0.452)、MOM4/4(0.060)、MOM7/2(0.013)；他们的最大回撤率分别为 9.78\%, 17.49\% and 9.52\%, 且都小于20\%;

>与benchmark相比较（夏普比率=0.231，最大回撤=2.347%），MOM2/9是表现最好的策略，夏普比率更高，风险更大。

<img src="5-s1.png" width="650" />
<img src="5-s2.png" width="650" />


计算资产净值（即我们投资组合的资产净值）也很重要，以衡量价格是否合理。 因此，资产净值为投资者的判断提供了一个重要而有用的衡量标准。 假设我们有本金 100,000 用于投资，我们为每个赢家和输家投资组合购买 20H（手）。 银行存款的无风险利率为0.3%。

MOM 2-9 
<img src="5-nav2.9.png" width="500" />

MOM 4-4 
<img src="5-nav4.4.png" width="500" />

MOM 7-2 
<img src="5-nav7.2.png" width="500" />

#### 5.3.4.2 不同时间段对比

不同产品包括农产品（Agricultural futures）、工业品（Metals futures）、能源化工产品（Energy/chemicals futures）、金融产品（Financial futures）。

#### 1. 5年期

<img src="5-full_perf.png" width="500" />

MOM 5年期
<img src="5-full_NAV.png" width="500" />

#### 2. 特殊时期：牛市 The Bull market

<img src="5-bull_perf.png" width="400" />

MOM 牛市
<img src="5-bull_NAV.png" width="500" />

#### 2. 特殊时期：疫情 COVID-19

<img src="5-covid_perf.png" width="500" />

MOM 疫情
<img src="5-covid_NAV.png" width="500" />
<img src="5-covid_规律.png" width="500" />

#### 5.3.4.2 产品类别对比


**四个子样本：**

1.不包括农产品的子样本

2.不包括能源/化学品的子样本

3.不包括金属的子样本

4.不包括金融期货的子样本

<br/>

**我们在以下两个时期测试上述四个子样本的表现：**
5年期：2015.01 - 2020.07

疫情 COVID-19：2019.12 - 2020.07

#### 1. 5年期

<img src="5-ex_full.png" width="800" />

#### 2. 特殊时期：疫情 COVID-19

<img src="5-ex_covid.png" width="800" />